# This is used for a test

In [1]:
import cv2, simplejson, os, shutil
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import random
import glob

In [2]:
fileName = '20170507T155114.708081'
fileNameVideo = fileName + '.avi'
vidcap = cv2.VideoCapture(fileNameVideo)

In [3]:
fileNameEvents = fileName + '-events.json'
events = simplejson.loads(open(fileNameEvents).read())
events

[{'end-frame': 111,
  'event-class': '0 added',
  'object-class': 'Clover Organic 2% Milk',
  'start-frame': 81},
 {'end-frame': 135,
  'event-class': '1 removed',
  'object-class': 'Clover Organic 2% Milk',
  'start-frame': 111},
 {'end-frame': 176,
  'event-class': '0 added',
  'object-class': 'Horizon Organic Chocolate Milk',
  'start-frame': 141},
 {'end-frame': 196,
  'event-class': '1 removed',
  'object-class': 'Horizon Organic Chocolate Milk',
  'start-frame': 176},
 {'end-frame': 228,
  'event-class': '0 added',
  'object-class': 'Horizon Organic Chocolate Milk',
  'start-frame': 208},
 {'end-frame': 264,
  'event-class': '1 removed',
  'object-class': 'Horizon Organic Chocolate Milk',
  'start-frame': 234}]

In [4]:
frames_start =[]
frames_end = []
labels = np.zeros(len(events))
for idx, event in enumerate(events):
    if event['event-class'] == '0 added':
        frames_start.append(event['start-frame'])
        frames_end.append(event['end-frame']-1)
        labels[idx] = 0
    elif event['event-class'] == '0 removed':
        frames_start.append(event['start-frame'])
        frames_end.append(event['end-frame']-1)
        labels[idx] = 1
    elif event['event-class'] == '1 added':
        frames_start.append(event['start-frame'])
        frames_end.append(event['end-frame']-1)
        labels[idx] = 2
    elif event['event-class'] == '1 removed':
        frames_start.append(event['start-frame'])
        frames_end.append(event['end-frame']-1)
        labels[idx] = 3
frames_start = np.array(frames_start)
frames_end = np.array(frames_end)
print(frames_start)
print(frames_end)
print(labels)

[ 81 111 141 176 208 234]
[110 134 175 195 227 263]
[ 0.  3.  0.  3.  0.  3.]


In [5]:
def get_frames(n, frames_start, frames_end):
    frames = np.zeros((n,frames_start.shape[0]))
    for i in range(frames_start.shape[0]):
        var = random.sample(range(int(frames_start[i]), int(frames_end[i])), n)
        var = np.sort(var)
        frames[:,i] = var
    frames = np.floor(frames)
    return frames
frames = get_frames(5, frames_start, frames_end)
print(frames)

[[  87.  113.  144.  182.  212.  240.]
 [  89.  116.  163.  183.  215.  247.]
 [  90.  124.  169.  184.  222.  252.]
 [  97.  127.  170.  189.  224.  253.]
 [ 100.  133.  173.  191.  225.  259.]]


In [6]:
frames_start

array([ 81, 111, 141, 176, 208, 234])

In [7]:
frames_end

array([110, 134, 175, 195, 227, 263])

In [8]:
num_frame_per_video = 5
num_sample_per_video = 5

input_width = 120
input_height = 120
dirname = 'Data'
categories = ['Add0', 'Remove0', 'Add1', 'Remove1']

data_len = frames_start.shape[0]
vidcap = cv2.VideoCapture(fileNameVideo)
if not os.path.isdir(dirname):
    os.mkdir(dirname)
for category in categories:
    if not os.path.isdir(dirname + '/' + category):
        os.mkdir(dirname + '/' + category)

event = 0
for j in range(num_sample_per_video):
    frames = get_frames(num_frame_per_video, frames_start, frames_end)
    for i in range(data_len):
        label = labels[i]
        category = categories[int(label)]
        
        vidcap = cv2.VideoCapture(fileNameVideo)
        success,image = vidcap.read()
        count_frame = 0
        count = 0
        while success:
            success,image = vidcap.read()
            if np.any(frames[:,i] == count_frame):
                image_crop = image[100:500, 150:800]
                image_down = cv2.resize(image_crop, dsize=(input_width, input_height), interpolation = cv2.INTER_CUBIC)
                imageFileName = '%s/%s/v_%s_e%s/f%d.jpg' % (dirname, category, category, event, count)
#                 cv2.imwrite(imageFileName, image_down) # save frame as JPEG file
                count += 1
            count_frame += 1
        
        vidcap.release()
        event += 1

In [9]:
# f = img.imread('%s/%s/v_%s_e%s_f%d.jpg' % (dirname, category, category, event-1, np.random.randint(0, num_frame_per_video)))
# plt.imshow(f)
# plt.show()

# Real data preprocessing starts from here

In [10]:
num_frame_per_video = 5
num_sample_per_video = 5

input_width = 128
input_height = 128
dirname = 'Data'
categories = ['Add0', 'Remove0', 'Add1', 'Remove1']

testList = open('myTest.list', 'w+')
trainList = open('myTrain.list', 'w+')

# N = len(glob.glob1('.', '*.avi'))
event_idx = 0
for filename in os.listdir('.'):
    
    if filename.endswith('.avi'): 
        fileName = os.path.splitext(filename)[0]
        
        # read json
        fileNameEvents = fileName + '-events.json'
        events = simplejson.loads(open(fileNameEvents).read())
        
        # store start/end frames and labels            
        frames_start =[]
        frames_end = []
        temp_label_list = []
        labels = np.zeros(len(events))
        for idx, event in enumerate(events):
            if event['event-class'] == '0 added':
                frames_start.append(event['start-frame'])
                frames_end.append(event['end-frame']-1)
                labels[idx] = 0
            elif event['event-class'] == '0 removed':
                frames_start.append(event['start-frame'])
                frames_end.append(event['end-frame']-1)
                labels[idx] = 1
            elif event['event-class'] == '1 added':
                frames_start.append(event['start-frame'])
                frames_end.append(event['end-frame']-1)
                labels[idx] = 2
            elif event['event-class'] == '1 removed':
                frames_start.append(event['start-frame'])
                frames_end.append(event['end-frame']-1)
                labels[idx] = 3
            
        frames_start = np.array(frames_start)
        frames_end = np.array(frames_end)
            
        # make a separate data folder
        if not os.path.isdir(dirname):
            os.mkdir(dirname)
        for category in categories:
            if not os.path.isdir(dirname + '/' + category):
                os.mkdir(dirname + '/' + category)
            
        # num sample per video
        for j in range(num_sample_per_video):
            # sample frame indices
            frames = get_frames(num_frame_per_video, frames_start, frames_end)
            for i in range(frames_start.shape[0]):
                label = labels[i]
                category = categories[int(label)]
        
                # load video
                fileNameVideo = fileName + '.avi'
                vidcap = cv2.VideoCapture(fileNameVideo)

                # data augmentation
                success,image = vidcap.read()
                count_frame = 0
                count = 0
                while success:
                    success,image = vidcap.read()
                    if np.any(frames[:,i] == count_frame):
                        # center crop
                        image_crop = image[100:500, 150:800]
                        # downsampling
                        image_down = cv2.resize(image_crop, dsize=(input_width, input_height), interpolation = cv2.INTER_CUBIC)
                        # write image
                        imageFileName = '%s/%s/v_%s_e%s_%d/%d.jpg' % (dirname, category, category, event_idx, j, count)
                        if not os.path.isdir('%s/%s/v_%s_e%s_%d' % (dirname, category, category, event_idx, j)):
                            os.mkdir('%s/%s/v_%s_e%s_%d' % (dirname, category, category, event_idx, j))
                        cv2.imwrite(imageFileName, image_down) # save frame as JPEG file
                        
                        imageFileName2 = '\\%s\\%s\\v_%s_e%s_%d %d' % (dirname, category, category, event_idx, j, label)
                        count += 1
                        
                    count_frame += 1
                
                if np.random.rand(1) < 0.85:
                    trainList.write(imageFileName2 + '\n')
                else:
                    testList.write(imageFileName2 + '\n')
                
                vidcap.release()
                event_idx += 1
        
    else:
        continue
        
testList.close()
trainList.close()

In [11]:
# testList.close()
# trainList.close()